In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from collections.abc import Sequence
import httpx
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
team_url = "https://www.transfermarkt.co.uk/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2020"

In [4]:
team_resp = httpx.get(
    team_url,
    headers={
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko gibi) Chrome/134.0.0.0 Safari/537.36'
    },
)

In [5]:
team_html = team_resp.content

In [6]:
page_soup = BeautifulSoup(team_html, "html.parser")

In [7]:
team_info = page_soup.find_all("td", {"class": "hauptlink no-border-links"})

In [8]:
team_name = [
    td.find("a").get("href").split("/")[1] if td.find("a") else None for td in team_info
]

In [9]:
team_id = [
    td.find("a").get("href").split("/")[4] if td.find("a") else None for td in team_info
]

In [10]:
urls = []
for td in team_info:
    data = td.find('a').get('href')
    team_name = data.split('/')[1]
    team_id = data.split('/')[4]
    year = data.split('/')[6]
    
    url = f'https://www.transfermarkt.co.uk/{team_name}/kader/verein/{team_id}/saison_id/{year}/plus/1'
    
    urls.append(url)
    

In [11]:
@dataclass
class Team:
    id: str
    name: str


class Parser(ABC):
    """ABC Protocol class for parsing data from transfermarkt."""

    @abstractmethod
    def parse(self, soup: BeautifulSoup) -> pd.DataFrame:
        pass


@dataclass
class Scraper:
    """Scrape data from transfermarkt for a given team and year."""

    team: Team
    parsers: Sequence[Parser]
    year: int
    url: str = (
        "https://www.transfermarkt.co.uk/{name}/kader/verein/{id}/saison_id/{year}/plus/1"
    )

    def run(self) -> pd.DataFrame:
        """Run the scraping process."""
        url = self.url.format(name=self.team.name, id=self.team.id, year=self.year)
        print(f"Scraping: {self.team.name} - {self.year}")

        soup = self._get_soup_content(url)  # get html content from url

        data = pd.concat(
            [parser.parse(soup) for parser in self.parsers], axis=1
        )  # concatenate parsers into a dataframe

        data["season"] = self.year  # add season to dataframe
        data["team"] = self.team.name  # add team name to dataframe

        return data

    def _get_soup_content(self, url: str) -> BeautifulSoup:
        """Get the html content from a given Transfermarkt url."""
        resp = self._make_request(url)
        return BeautifulSoup(resp.content, "html.parser")

    def _make_request(self, url: str) -> httpx.Response:
        """Make a request to a given Transfermarkt url."""
        try:
            response = httpx.get(
                url,
                headers={
                    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko gibi) Chrome/134.0.0.0 Safari/537.36'
                },
                timeout=60,
            )
            response.raise_for_status()
            return response

        except httpx.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            raise e

In [12]:
class PlayerNames(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        elements = soup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        names = [td.get("title") if td.get("title") else None for td in elements]
        return pd.Series(names, name="name")

In [13]:
class PlayerNumbers(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        numbers = [stat for stat in stats[0::8]]
        numbers = [
            (
                td.find("div", class_="rn_nummer").text.strip()
                if td.find("div", class_="rn_nummer")
                else None
            )
            for td in numbers
        ]
        return pd.Series(numbers, name="number")

In [14]:
class PlayerAges(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.DataFrame:
        stats = soup.find_all("td", {"class": "zentriert"})
        ages = [stat for stat in stats[1::8]]
        dob = [td.text.strip().split(" (")[0] if td.text else None for td in ages]
        age = [
            int(td.text.strip().split(" (")[1].split(")")[0]) if td.text else None
            for td in ages
        ]
        return pd.DataFrame({"dob": dob, "age": age})

In [15]:
class PlayerCountries(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        countries = [stat for stat in stats[2::8]]
        countries = [
            td.find("img").get("title") if td.find("img") else None for td in countries
        ]
        return pd.Series(countries, name="country")

In [16]:
class CurrentClubs(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        current_clubs = [stat for stat in stats[3::8]]
        current_clubs = [
            td.find("a").get("title") if td.find("a") else None for td in current_clubs
        ]
        return pd.Series(current_clubs, name="current_club")

In [17]:
class PlayerHeights(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        heights = [stat for stat in stats[4::8]]
        heights = [td.text if td.text else None for td in heights]
        return pd.Series(heights, name="height")

In [18]:
class PlayerFoot(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        foots = [stat for stat in stats[5::8]]
        foots = [td.text if td.text else None for td in foots]
        return pd.Series(foots, name="foot")

In [19]:
class PlayerJoinedDate(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        joined_date = [stat for stat in stats[6::8]]
        joined_date = [td.text if td.text else None for td in joined_date]
        return pd.Series(joined_date, name="joined_date")

In [20]:
class PlayerSigningFee(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        signing_info = [stat for stat in stats[7::8]]
        signing_fee = [
            td.find("a").get("title").split(": Ablöse ")[1] if td.find("a") else 0
            for td in signing_info
        ]
        return pd.Series(signing_fee, name="signing_fee")

In [21]:
class PlayerSignedFrom(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        stats = soup.find_all("td", {"class": "zentriert"})
        signing_info = [stat for stat in stats[7::8]]
        signed_from = [
            td.find("a").get("title").split(": Ablöse ")[0] if td.find("a") else None
            for td in signing_info
        ]
        return pd.Series(signed_from, name="signed_from")

In [22]:
class PlayerValues(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        values = soup.find_all("td", {"class": "rechts hauptlink"})
        values = [td.find("a").text if td.find("a") else "€0" for td in values]
        return pd.Series(values, name="value")

In [23]:
class PlayerPositions(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        pos_soup = soup.find_all("td", {"class": "posrela"})
        positions = [
            td.find_all("tr")[1].find("td").text.strip() if td.find_all("tr") else None
            for td in pos_soup
        ]
        return pd.Series(positions, name="position")

In [24]:
class TransfermarktName(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        links = soup.find_all("td", {"class": "hauptlink"})
        tm_name = [
            link.find("a")["href"].split("/")[1] if link.find("a") else None
            for link in links[::2]
        ]
        return pd.Series(tm_name, name="tm_name")

In [25]:
class TransfermarktId(Parser):
    def parse(self, soup: BeautifulSoup) -> pd.Series:
        links = soup.find_all("td", {"class": "hauptlink"})
        tm_id = [
            link.find("a")["href"].split("/")[4] if link.find("a") else None
            for link in links[::2]
        ]
        return pd.Series(tm_id, name="tm_id")

In [26]:
parsers = (
    PlayerNames(),
    PlayerNumbers(),
    PlayerAges(),
    PlayerCountries(),
    CurrentClubs(),
    PlayerHeights(),
    PlayerFoot(),
    PlayerJoinedDate(),
    PlayerSigningFee(),
    PlayerSignedFrom(),
    PlayerValues(),
    PlayerPositions(),
    TransfermarktName(),
    TransfermarktId()
)

In [27]:
def get_team_info(league: str, league_id: str, year: int) -> tuple:
    link = "https://www.transfermarkt.co.uk/{league}/startseite/wettbewerb/{league_id}/plus/?saison_id={year}"
    url = link.format(league=league, league_id=league_id, year=year)
    resp = httpx.get(
        url,
        headers={
            "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, Gecko gibi) Chrome/134.0.0.0 Safari/537.36'
        },
        timeout=20,
    )
    soup = BeautifulSoup(resp.content, "html.parser")
    team_info = soup.find_all("td", {"class": "hauptlink no-border-links"})
    team_name = [td.find('a').get('href').split('/')[1] for td in team_info]
    team_id = [td.find('a').get('href').split('/')[4] for td in team_info]
    return tuple(zip(team_name, team_id))

In [28]:
pl_teams = get_team_info('ligue-1', 'FR1', 2020)

In [29]:
teams = [Team(id=id, name=name) for name, id in zip(team_name, team_id)]

In [30]:
teams

[Team(id='1', name='n'),
 Team(id='1', name='i'),
 Team(id='6', name='m'),
 Team(id='0', name='e')]

In [31]:
dfs = []
for name, id_ in pl_teams:
    team = Team(id=id_, name=name)
    scraper = Scraper(team=team, parsers=parsers, year=2020)
    df = scraper.run()
    dfs.append(df)
    time.sleep(5) # sleep for 5 seconds to avoid getting blocked

Scraping: fc-paris-saint-germain - 2020
Scraping: olympique-lyon - 2020
Scraping: as-monaco - 2020
Scraping: losc-lille - 2020
Scraping: fc-stade-rennes - 2020
Scraping: olympique-marseille - 2020
Scraping: ogc-nizza - 2020
Scraping: as-saint-etienne - 2020
Scraping: montpellier-hsc - 2020
Scraping: sco-angers - 2020
Scraping: rc-strassburg-alsace - 2020
Scraping: fc-girondins-bordeaux - 2020
Scraping: stade-reims - 2020
Scraping: rc-lens - 2020
Scraping: fc-metz - 2020
Scraping: stade-brest-29 - 2020
Scraping: fc-nantes - 2020
Scraping: fc-lorient - 2020
Scraping: dijon-fco - 2020
Scraping: nimes-olympique - 2020


In [32]:
data = pd.concat(dfs)

In [33]:
data

,name,number,dob,age,country,current_club,height,foot,joined_date,signing_fee,signed_from,value,position,tm_name,tm_id,season,team
0,Keylor Navas,1,"Dec 15, 1986",34,Costa Rica,CA Newell's Old Boys,"1,85m",right,"Sep 2, 2019",€15.00m,Real Madrid,€12.00m,Goalkeeper,keylor-navas,79422,2020,fc-paris-saint-germain
1,Sergio Rico,16,"Sep 1, 1993",27,Spain,Al-Gharafa SC,"1,96m",right,"Sep 5, 2020",€6.00m,Sevilla FC,€7.00m,Goalkeeper,sergio-rico,207302,2020,fc-paris-saint-germain
2,Marcin Bulka,-,"Oct 4, 1999",21,Poland,OGC Nice,"1,99m",right,"Jul 6, 2019",free transfer,Chelsea FC U23,€800k,Goalkeeper,marcin-bulka,465955,2020,fc-paris-saint-germain
3,Alexandre Letellier,30,"Dec 11, 1990",30,France,Without ClubWithout Club,"1,93m",right,"Sep 25, 2020",free transfer,US Orléans,€400k,Goalkeeper,alexandre-letellier,93730,2020,fc-paris-saint-germain
4,Garissone Innocent,-,"Apr 16, 2000",21,Haiti,FK Riteriai,"1,92m",right,"Jul 1, 2019",-,Paris Saint-Germain U19,€300k,Goalkeeper,garissone-innocent,460626,2020,fc-paris-saint-germain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,Karim Aribi,13,"Jun 24, 1994",27,Algeria,MC Oran,"1,92m",right,"Oct 2, 2020",€1.00m,Etoile Sportive du Sahel,€800k,Centre-Forward,karim-aribi,456080,2020,nimes-olympique
32,Nolan Roux,25,"Mar 1, 1988",33,France,None,"1,82m",right,"Jan 30, 2020",free transfer,EA Guingamp,€800k,Centre-Forward,nolan-roux,81460,2020,nimes-olympique
33,Clément Depres,9,"Nov 25, 1994",26,France,Ratchaburi FC,"1,88m",right,"Jul 1, 2015",-,Nîmes Olympique B,€600k,Centre-Forward,clement-depres,347595,2020,nimes-olympique
34,Sami Ben Amar,24,"Mar 2, 1998",23,Morocco,FC Bourgoin-Jallieu,"1,84m",right,"Jul 1, 2018",-,Nîmes Olympique B,€350k,Centre-Forward,sami-ben-amar,524318,2020,nimes-olympique


In [34]:
# Export the data DataFrame to Excel
data.to_excel("../Datas/my_2020_ligue_1_data.xlsx", index=False)